# Tools

The tool abstraction in LangChain associates a Python function with a schema that defines the function's name, description and expected arguments

The key attributes that correspond to the tool's schema:

- name: The name of the tool.

- description: A description of what the tool does.

- args: Property that returns the JSON schema for the tool's arguments.


The key methods to execute the function associated with the tool:

- invoke: Invokes the tool with the given arguments.

- ainvoke: Invokes the tool with the given arguments, asynchronously. Used for async programming with Langchain.

### How to create tools using the @tool decorator

In [7]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [8]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True


### How to create tools with StructuredTool

In [9]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b



multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="multiply", 
    args_schema= CalculatorInput,
    return_direct=True
    #coroutine= for async tool
    )


add_tool = StructuredTool.from_function(
    func=add,
    name="add", 
    args_schema= CalculatorInput,
    return_direct=True
    #coroutine= for async tool
    )

### Raising errors

In [10]:
from langchain_core.tools import ToolException


def get_weather(city: str) -> int:
    """Get weather for the given city."""
    raise ToolException(f"Error: There is no city by the name of {city}.")


get_weather_tool = StructuredTool.from_function(
    func=get_weather,
    handle_tool_error=True, # or a string 
)

get_weather_tool.invoke({"city": "foobar"})

'Error: There is no city by the name of foobar.'

### Best practices
When designing tools to be used by models, keep the following in mind:

* Tools that are well-named, correctly-documented and properly type-hinted are easier for models to use.
* Design simple and narrowly scoped tools, as they are easier for models to use correctly.
* Use chat models that support tool-calling APIs to take advantage of tools.
* Asking the model to select from a large list of tools poses challenges for the model.

### Binding tools to LLMs

![alt text](tool_call-8d4a8b18e90cacd03f62e94071eceace.png)

In [11]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    google_api_key= os.getenv("GOOGLE_API_KEY")
)

In [12]:
tools = [add_tool,multiply_tool]

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12? Also, what is 11 + 49"

response = llm_with_tools.invoke(query)

response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '95009fcb-89e0-41c0-902d-dc521eaf3661',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': 'be0dd173-27a0-4110-980d-ce8774289655',
  'type': 'tool_call'}]

### Passing the tool output back to the model

![alt text](tool_invocation-7f277888701ee431a17607f1a035c080.png)

![alt text](tool_results-71b4b90f33a56563c102d91e7821a993.png)

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query)]

messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = {"add": add_tool, "multiply": multiply_tool}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run-588a0bd7-9640-4d0c-8c3a-a216b4960461-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '95009fcb-89e0-41c0-902d-dc521eaf3661', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': 'be0dd173-27a0-4110-980d-ce8774289655', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 10, 'total_tokens': 94, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='36', name='multiply', tool_call_id='95009fcb-89e0-41c0-902d-dc521eaf3661'),
 ToolMessage(content='60', name='add', tool_call_id='be0dd173-27a0-4110-980d-ce8774289655')

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 = 36. 11 + 49 = 60.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run-0e6d82fa-fad0-4426-a551-22db6a433e22-0', usage_metadata={'input_tokens': 100, 'output_tokens': 23, 'total_tokens': 123, 'input_token_details': {'cache_read': 0}})

### Tool Artifacts

Outputs are designed to be fed back to a model. However, there are artifacts of a tool's execution that we want to make accessible to downstream

In [15]:
import random
from typing import List, Tuple

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """Generate size random ints in the range [min, max]."""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"Successfully generated array of {size} random ints in [{min}, {max}]."
    return content, array

In [16]:
llm_with_tools = llm.bind_tools([generate_random_ints])

ai_msg = llm_with_tools.invoke("generate 6 positive ints less than 25")
ai_msg.tool_calls

[{'name': 'generate_random_ints',
  'args': {'size': 6.0, 'min': 1.0, 'max': 24.0},
  'id': 'edd40db0-e68c-4b01-9c8d-3de3ee68dadb',
  'type': 'tool_call'}]

In [17]:
generate_random_ints.invoke(ai_msg.tool_calls[0])

ToolMessage(content='Successfully generated array of 6 random ints in [1, 24].', name='generate_random_ints', tool_call_id='edd40db0-e68c-4b01-9c8d-3de3ee68dadb', artifact=[22, 12, 23, 3, 5, 9])